In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, \
    OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
from google.colab import drive
drive.mount('/content/drive')
PATH = r"/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv(PATH+'DPE_train.csv',index_col=0)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,numero_dpe,tr001_modele_dpe_id,consommation_energie,classe_consommation_energie,tr002_type_batiment_id,annee_construction,surface_habitable,tv016_departement_id,commune,code_insee_commune_corrige,shon,date_reception_dpe,besoin_chauffage,deperdition_enveloppe,deperdition_renouvellement_air,tv026_classe_inertie_id,altitude,nombre_niveau,hsp_moyenne
0,1354V1000001@,1,282.00,E,1,1948,118.00,54,PEXONNE,54540,118.00,2013-04-08 04:00:00,338.36,401.97,56.03,5.0,450,2.0,2.50
1,1379V1000001G,1,326.00,E,1,1983,69.84,79,LA MOTHE-SAINT-HÉRAY,79184,69.84,2013-04-08 04:00:00,131.98,172.08,58.56,3.0,1,1.0,2.64
2,1313V1000001U,1,206.00,D,2,2006,84.00,13,MARSEILLE,13013,84.00,2013-04-08 04:00:00,96.69,164.86,67.56,5.0,520,1.0,2.50
3,1380V1000001Y,1,200.00,D,1,2001,99.00,80,VAIRE-SOUS-CORBIE,80800,99.00,2013-04-08 04:00:00,111.74,177.31,44.77,3.0,110,1.0,2.40
4,1392L1000001A,9,242.02,E,2,1980,64.05,92,COURBEVOIE,92026,0.00,2013-04-08 04:00:00,64.46,98.04,45.47,1.0,40,1.0,2.50


In [4]:
df.classe_consommation_energie.value_counts()

D    714181
E    563230
C    327682
F    227938
G     81873
B     62783
A     22309
N         5
Name: classe_consommation_energie, dtype: int64

In [5]:
# discard misclassified elements (we cannot know if the class is right but the energy consumption is wrong and vice versa)
wrong_class_A=(df.classe_consommation_energie=='A')&((df.consommation_energie<0) | (df.consommation_energie>50))
df=df.loc[~wrong_class_A]

wrong_class_B=(df.classe_consommation_energie=='B')&((df.consommation_energie<51) | (df.consommation_energie>90))
df=df.loc[~wrong_class_B]

wrong_class_C=(df.classe_consommation_energie=='C')&((df.consommation_energie<91) | (df.consommation_energie>150))
df=df.loc[~wrong_class_C]

wrong_class_D=(df.classe_consommation_energie=='D')&((df.consommation_energie<151) | (df.consommation_energie>230))
df=df.loc[~wrong_class_D]

wrong_class_E=(df.classe_consommation_energie=='E')&((df.consommation_energie<231) | (df.consommation_energie>330))
df=df.loc[~wrong_class_E]

wrong_class_F=(df.classe_consommation_energie=='F')&((df.consommation_energie<331) | (df.consommation_energie>450))
df=df.loc[~wrong_class_F]

wrong_class_G=(df.classe_consommation_energie=='G')&((df.consommation_energie<450))
df=df.loc[~wrong_class_G]

In [6]:
df=df.loc[df.classe_consommation_energie!='N']

In [7]:
num_cols= list(df.select_dtypes(include=np.number).columns)
corr_matrix_num_target=df[[*num_cols]].corr()
corr_matrix_num_target["consommation_energie"].sort_values(ascending=False)

consommation_energie              1.000000
tv016_departement_id              0.000896
tr002_type_batiment_id            0.000717
tr001_modele_dpe_id               0.000627
annee_construction                0.000617
altitude                          0.000151
besoin_chauffage                  0.000018
nombre_niveau                    -0.000081
deperdition_enveloppe            -0.000181
shon                             -0.000314
surface_habitable                -0.000873
deperdition_renouvellement_air   -0.000978
hsp_moyenne                      -0.001316
tv026_classe_inertie_id          -0.008814
Name: consommation_energie, dtype: float64

In [8]:
df.columns

Index(['numero_dpe', 'tr001_modele_dpe_id', 'consommation_energie',
       'classe_consommation_energie', 'tr002_type_batiment_id',
       'annee_construction', 'surface_habitable', 'tv016_departement_id',
       'commune', 'code_insee_commune_corrige', 'shon', 'date_reception_dpe',
       'besoin_chauffage', 'deperdition_enveloppe',
       'deperdition_renouvellement_air', 'tv026_classe_inertie_id', 'altitude',
       'nombre_niveau', 'hsp_moyenne'],
      dtype='object')

In [9]:
cols_keep=['altitude', 'annee_construction', 'besoin_chauffage','date_reception_dpe', 'hsp_moyenne', 'nombre_niveau',
          'surface_habitable', 'tr001_modele_dpe_id', 'tr002_type_batiment_id',
          'tv016_departement_id', 'tv026_classe_inertie_id']

df1=df[cols_keep]

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1983580 entries, 0 to 2000000
Data columns (total 11 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   altitude                 int64  
 1   annee_construction       int64  
 2   besoin_chauffage         float64
 3   date_reception_dpe       object 
 4   hsp_moyenne              float64
 5   nombre_niveau            float64
 6   surface_habitable        float64
 7   tr001_modele_dpe_id      int64  
 8   tr002_type_batiment_id   int64  
 9   tv016_departement_id     int64  
 10  tv026_classe_inertie_id  float64
dtypes: float64(5), int64(5), object(1)
memory usage: 181.6+ MB


In [11]:
(df1.isna().sum()).sort_values(ascending=False)

tv026_classe_inertie_id    368425
nombre_niveau                  48
surface_habitable              23
altitude                        0
annee_construction              0
besoin_chauffage                0
date_reception_dpe              0
hsp_moyenne                     0
tr001_modele_dpe_id             0
tr002_type_batiment_id          0
tv016_departement_id            0
dtype: int64

In [12]:
class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X, y):

        #numeric transformer
        numeric_transformer=SimpleImputer(strategy='median')

        #date transformer
        def process_date(X):
            date = pd.to_datetime(X['date_reception_dpe'])
            return np.c_[(date-min(date)).dt.total_seconds()]

        date_transformer = FunctionTransformer(process_date)

        #label transformer

        label_transformer = LabelEncoder()

        num_cols_w_nan=['tv026_classe_inertie_id','surface_habitable','nombre_niveau']
        date_col=['date_reception_dpe']

        drop_cols=['consommation_energie', 'deperdition_enveloppe', 'deperdition_renouvellement_air', 'code_insee_commune_corrige', 'commune', 'numero_dpe','shon']

        preprocessor = ColumnTransformer(
            transformers=[('date', make_pipeline(date_transformer,SimpleImputer(strategy='median')), date_col),
                
                ('num',make_pipeline(numeric_transformer,StandardScaler()), num_cols_w_nan),

                ('drop cols', 'drop', drop_cols),
            ],remainder='passthrough')
        self.preprocessor = preprocessor
        self.preprocessor.fit(X)

        self.labelencoder=label_transformer
        self.labelencoder.fit(y)
        return self

    def transform(self, X,y):
        return self.preprocessor.transform(X),self.labelencoder.transform(y)

# Classifier

In [15]:
%%file submissions/starting_kit/classifier.py
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, \
    OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X, y):

        #numeric transformer
        numeric_transformer=SimpleImputer(strategy='median')

        #date transformer
        def process_date(X):
            date = pd.to_datetime(X['date_reception_dpe'])
            return np.c_[(date-min(date)).dt.total_seconds()]

        date_transformer = FunctionTransformer(process_date)

        #label transformer

        label_transformer = LabelEncoder()

        num_cols_w_nan=['tv026_classe_inertie_id','surface_habitable','nombre_niveau']
        date_col=['date_reception_dpe']

        drop_cols=['consommation_energie', 'deperdition_enveloppe', 'deperdition_renouvellement_air', 'code_insee_commune_corrige', 'commune', 'numero_dpe','shon']

        preprocessor = ColumnTransformer(
            transformers=[('date', make_pipeline(date_transformer,SimpleImputer(strategy='median')), date_col),
                
                ('num',make_pipeline(numeric_transformer,StandardScaler()), num_cols_w_nan),

                ('drop cols', 'drop', drop_cols),
            ],remainder='passthrough')
        self.preprocessor = preprocessor
        self.preprocessor.fit(X)

        self.labelencoder=label_transformer
        self.labelencoder.fit(y)
        return self

    def transform(self, X,y):
        return self.preprocessor.transform(X),self.labelencoder.transform(y)

class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = Pipeline([
            ('classifier', RandomForestClassifier())
        ])
        self.fe = FeatureExtractor()

    def fit(self, X, y):
        self.fe.fit(X,y)
        Xf,yf=self.fe.transform(X,y)
        self.clf.fit(Xf, yf)

    def predict_proba(self, X):
        Xf=self.fe.transform(X)
        return self.clf.predict_proba(Xf)

In [ ]:
classifier = Classifier()
classifier.fit(df.drop(columns=['classe_consommation_energie'])[200000:],df['classe_consommation_energie'][200000:])

The metric of the challenge will be MSE.

In [ ]:
y_pred = classifier.predict_proba(df.drop(columns=['classe_consommation_energie'])[-200000:])

In [ ]:
np.argmax(y_pred,1).shape

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['classe_consommation_energie'][-200000:], np.argmax(y_pred,1))

You can test locally your submission before sending it to the RAMP platform backend using the command !ramp_test_submission --submission starting_kit.

The usual way to use this is to explore various models, feature transformations, hyper-parameters optimization and test them using the following command. It uses our metric to estimate your score.

To test files stored in a different folder, just replace 'starting_kit' by the name of your folder

In [ ]:
!ramp_test_submission --ramp_data_dir './data/' --submission starting_kit